In [2]:
import pandas as pd

In [5]:
def get_year(year):
    df = pd.read_csv('data/dk_%s_raw.csv.gz' % year, encoding='latin-1', delimiter=';', compression='gzip')
    df['year'] = year
    return df

df = pd.concat([get_year(year) for year in range(2014, 2016)])
df.head()

/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (2,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Navn,Kommune,Postnr,Region,Measure,EU(DKK),National(DKK),Sum(DKK),Totalt(DKK),Unnamed: 9,year
0,Videncentret for Landbrug,751,8200,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,14.442.555,14.442.555,28.885.110,52.212.463,NaN,2014
1,Mariagerfjord kommune,Rebild Kommune,9500,Region Nordjylland,(a) measures targeting the sustainable use of ...,19.811.565,6.603.855,26.415.420,27.130.177,NaN,2014
2,Videncentret for Landbrug,751,8200,Region Midtjylland,(b)measures aimed at restructuring and develop...,11.505.288,11.505.288,23.010.576,52.212.463,NaN,2014
3,NaturErhvervstyrelsen,Københavns Kommune,1602,Region Hovedstaden,Technical assistance,21.800.000,0,21.800.000,22.056.850,NaN,2014
4,Økologisk Landsforening,751,8230,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,7.275.243,7.275.243,14.550.486,15.441.050,NaN,2014


In [6]:
df['year'].value_counts()

2015    136704
2014    133122
Name: year, dtype: int64

In [7]:
import re
from slugify import slugify

NUM_RE = re.compile('\[Anonym - (\d+)\]')
get_id = lambda x: NUM_RE.match(x).group(1)
df['recipient_id'] = df.apply(lambda x: 'DK-%s' % get_id(x['Navn']) if x['Navn'].startswith('[Anonym - ') else
                                        'DK-%s-%s' % (x['Postnr'], slugify(x['Navn'])), 1)
df['Navn'] = df['Navn'].apply(lambda x: '' if x.startswith('[Anonym - ') else x)
df.head()

,Navn,Kommune,Postnr,Region,Measure,EU(DKK),National(DKK),Sum(DKK),Totalt(DKK),Unnamed: 9,year,recipient_id
0,Videncentret for Landbrug,751,8200,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,14.442.555,14.442.555,28.885.110,52.212.463,NaN,2014,DK-8200-Videncentret-for-Landbrug
1,Mariagerfjord kommune,Rebild Kommune,9500,Region Nordjylland,(a) measures targeting the sustainable use of ...,19.811.565,6.603.855,26.415.420,27.130.177,NaN,2014,DK-9500-Mariagerfjord-kommune
2,Videncentret for Landbrug,751,8200,Region Midtjylland,(b)measures aimed at restructuring and develop...,11.505.288,11.505.288,23.010.576,52.212.463,NaN,2014,DK-8200-Videncentret-for-Landbrug
3,NaturErhvervstyrelsen,Københavns Kommune,1602,Region Hovedstaden,Technical assistance,21.800.000,0,21.800.000,22.056.850,NaN,2014,DK-1602-NaturErhvervstyrelsen
4,Økologisk Landsforening,751,8230,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,7.275.243,7.275.243,14.550.486,15.441.050,NaN,2014,DK-8230-Okologisk-Landsforening


In [8]:
df['recipient_location'] = df.apply(lambda x: '%s, %s' % (x['Kommune'], x['Region']), 1)

df = df.rename(columns={
    'Navn': 'recipient_name',
    'Postnr': 'recipient_postcode',
    'Measure': 'scheme',
    'EU(DKK)': 'amount'
})

df['currency'] = 'DKK'
df['country'] = 'DK'
df['year'] = 2014
df.head()

,recipient_name,Kommune,recipient_postcode,Region,scheme,amount,National(DKK),Sum(DKK),Totalt(DKK),Unnamed: 9,year,recipient_id,recipient_location,currency,country
0,Videncentret for Landbrug,751,8200,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,14.442.555,14.442.555,28.885.110,52.212.463,NaN,2014,DK-8200-Videncentret-for-Landbrug,"751, Region Midtjylland",DKK,DK
1,Mariagerfjord kommune,Rebild Kommune,9500,Region Nordjylland,(a) measures targeting the sustainable use of ...,19.811.565,6.603.855,26.415.420,27.130.177,NaN,2014,DK-9500-Mariagerfjord-kommune,"Rebild Kommune, Region Nordjylland",DKK,DK
2,Videncentret for Landbrug,751,8200,Region Midtjylland,(b)measures aimed at restructuring and develop...,11.505.288,11.505.288,23.010.576,52.212.463,NaN,2014,DK-8200-Videncentret-for-Landbrug,"751, Region Midtjylland",DKK,DK
3,NaturErhvervstyrelsen,Københavns Kommune,1602,Region Hovedstaden,Technical assistance,21.800.000,0,21.800.000,22.056.850,NaN,2014,DK-1602-NaturErhvervstyrelsen,"Københavns Kommune, Region Hovedstaden",DKK,DK
4,Økologisk Landsforening,751,8230,Region Midtjylland,(a) measures aimed at promoting knowledge and ...,7.275.243,7.275.243,14.550.486,15.441.050,NaN,2014,DK-8230-Okologisk-Landsforening,"751, Region Midtjylland",DKK,DK


In [9]:
df = df.drop(['Kommune', 'Region', 'National(DKK)', 'Sum(DKK)', 'Totalt(DKK)', 'Unnamed: 9'], 1)
df.head()

,recipient_name,recipient_postcode,scheme,amount,year,recipient_id,recipient_location,currency,country
0,Videncentret for Landbrug,8200,(a) measures aimed at promoting knowledge and ...,14.442.555,2014,DK-8200-Videncentret-for-Landbrug,"751, Region Midtjylland",DKK,DK
1,Mariagerfjord kommune,9500,(a) measures targeting the sustainable use of ...,19.811.565,2014,DK-9500-Mariagerfjord-kommune,"Rebild Kommune, Region Nordjylland",DKK,DK
2,Videncentret for Landbrug,8200,(b)measures aimed at restructuring and develop...,11.505.288,2014,DK-8200-Videncentret-for-Landbrug,"751, Region Midtjylland",DKK,DK
3,NaturErhvervstyrelsen,1602,Technical assistance,21.800.000,2014,DK-1602-NaturErhvervstyrelsen,"Københavns Kommune, Region Hovedstaden",DKK,DK
4,Økologisk Landsforening,8230,(a) measures aimed at promoting knowledge and ...,7.275.243,2014,DK-8230-Okologisk-Landsforening,"751, Region Midtjylland",DKK,DK


In [10]:
len(df)

269826

In [12]:
df['amount'] = pd.to_numeric(df['amount'].str.replace('.', '').str.replace(',', '.'))

In [13]:
df['amount'].sum()

14150473330.0

In [14]:
df.to_csv('dk_2014_2015.csv.gz', index=False, encoding='utf-8', compression='gzip')